<a href="https://colab.research.google.com/github/kviercz/projects/blob/main/aa511_assignment2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import itertools
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [2]:
# 1. Load Dataset
df = pd.read_csv('M2-Advertising Dataset.csv')

In [3]:
# 2. Data preprocessing

# Get info about the dataset and check for null values
print(df.describe())
df.info()
df.isnull().sum()

# Convert the countries column into categorical values to improve data training
df = pd.get_dummies(df, columns=['Country'], drop_first=True)

# Try to use data processing to handle the ad topic
tfidf = TfidfVectorizer(stop_words='english', max_features=50)  # Limit to top 50 features
ad_topic_features = tfidf.fit_transform(df['Ad Topic Line']).toarray()
ad_topic_df = pd.DataFrame(ad_topic_features, columns=tfidf.get_feature_names_out())

# Concatenate TF-IDF features with the original dataframe
df = pd.concat([df, ad_topic_df], axis=1)

# Drop the original 'Ad Topic Line' column
df = df.drop(columns=['Ad Topic Line'])

# Handle non-numeric columns
df.drop(columns=['Timestamp'], inplace=True)
df = pd.get_dummies(df, columns=['City'])


       Daily Time Spent on Site          Age   Area Income  \
count               1000.000000  1000.000000   1000.000000   
mean                  65.000200    36.009000  55000.000080   
std                   15.853615     8.785562  13414.634022   
min                   32.600000    19.000000  13996.500000   
25%                   51.360000    29.000000  47031.802500   
50%                   68.215000    35.000000  57012.300000   
75%                   78.547500    42.000000  65470.635000   
max                   91.430000    61.000000  79484.800000   

       Daily Internet Usage         Male  Clicked on Ad  
count           1000.000000  1000.000000     1000.00000  
mean             180.000100     0.481000        0.50000  
std               43.902339     0.499889        0.50025  
min              104.780000     0.000000        0.00000  
25%              138.830000     0.000000        0.00000  
50%              183.130000     0.000000        0.50000  
75%              218.792500     1.0

In [4]:
# 3. Split data into train and test sets
X = df.drop('Clicked on Ad', axis=1)
y = df['Clicked on Ad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
# 4. Scale the data using Standard Scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# 5. Build an ANN Classification Model
# This function takes in different parameters to experiment with different
# architectures and parameters. The function returns a model that can be used
# to evaluate performance
def build_model(hidden_layers, neurons, activation, dropout_rate, \
                learning_rate):
    model = keras.Sequential()
    model.add(keras.layers.Dense(neurons[0], activation=activation, \
                                 input_shape=[X_train.shape[1]]))

    for i in range(1, hidden_layers):
        model.add(keras.layers.Dense(neurons[i], activation=activation))
        model.add(keras.layers.Dropout(dropout_rate))

    # Binary classificiation output layer
    model.add(keras.layers.Dense(1))

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

  # model.fit(X_train, y_train, epochs=100, batch_size=16)
  # test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print('Test Loss: {:.4f}'.format(test_loss))

In [10]:
# 6 Different Architectures
# Using the function above, experiment with different architectures and parameters
# all the models will be evaluated using the same metrics for an easier comparison

# Experiment with different architectures and hyperparameters
hyper_parameters = {
    "hidden_layers": [2,3],
    "neurons": [[64, 32], [128, 64, 32]],
    "activation": ["relu", "tanh"],
    "dropout_rate": [0.2, 0.5],
    "learning_rate": [0.001, 0.01]
}
# Initialize results list to store the metrics for each model
results = []

# Using itertools to iterate through all the combinations of hyperparameters
# instead of lots of nested for loops.
for params in itertools.product(*hyper_parameters.values()):
    params = dict(zip(hyper_parameters.keys(), params))

    if len(params['neurons']) == params['hidden_layers']:
        # Build model with given hyperparameters
        model = build_model(**params)
        model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
        y_pred_proba = model.predict(X_test)
        y_pred = (y_pred_proba > 0.5).astype(int)

        # Collect performance metrics for this model
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)

        # Save results to results list for comparison
        results.append({**params,
                        "test_lost": test_loss,
                        "test_accuracy": test_accuracy,
                        "accuracy": accuracy,
                        "precision": precision,
                        "recall": recall,
                        "f1": f1,
                        "roc_auc": roc_auc})

# Analyze the results (e.g., sort by a metric, print top performers)
results_df = pd.DataFrame(results)
print(results_df.sort_values('roc_auc', ascending=False))

# Experiment with different regularization techniques

# print('Test Loss: {:.4f}'.format(test_loss))
# print('Test Accuracy: {:.4f}'.format(test_accuracy))

# # ROC AUC
# y_pred_prob = model.predict(X_test)
# roc_auc = roc_auc_score(y_test, y_pred_prob)
# print('ROC AUC: {:.4f}'.format(roc_auc))

# # Predictive classes
# y_pred = (y_pred_prob > 0.5).astype(int)

# # Accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print('Accuracy: {:.4f}'.format(accuracy))

# # Precision
# precision = precision_score(y_test, y_pred)
# print('Precision: {:.4f}'.format(precision))

# # Recall
# recall = recall_score(y_test, y_pred)
# print('Recall: {:.4f}'.format(recall))

# # F1
# f1 = f1_score(y_test, y_pred)
# print('F1 Score: {:.4f}'.format(f1))

7/7 [==============================] - 0s 3ms/step
    hidden_layers        neurons activation  dropout_rate  learning_rate  \
3               2       [64, 32]       relu           0.5          0.010   
11              3  [128, 64, 32]       relu           0.5          0.010   
5               2       [64, 32]       tanh           0.2          0.010   
15              3  [128, 64, 32]       tanh           0.5          0.010   
1               2       [64, 32]       relu           0.2          0.010   
7               2       [64, 32]       tanh           0.5          0.010   
10              3  [128, 64, 32]       relu           0.5          0.001   
8               3  [128, 64, 32]       relu           0.2          0.001   
14              3  [128, 64, 32]       tanh           0.5          0.001   
2               2       [64, 32]       relu           0.5          0.001   
12              3  [128, 64, 32]       tanh           0.2          0.001   
0               2       [64, 32]     